In [623]:
import numpy as np
import pandas as pd
import xgboost as xgb

train_df = pd.read_csv('data/train.csv', index_col='Id')
test_df = pd.read_csv('data/test.csv', index_col='Id')

In [624]:
test_df['SalePrice'] = 0

In [625]:
all_data = pd.concat([train_df, test_df], axis=0)

In [626]:
# ВАЖНО ЧТОБЫ ПРИЗНАКИ БЫЛИ НЕЛИНЕЙНО ЗАВИСИМЫЕ
all_data["Lack_of_feature_index"] = all_data.isnull().sum(axis=1)  # Кол-во NaN значений. Нелинейный признак
all_data['Has_Alley'] = all_data["Alley"].notnull()  # НАЛИЧИЕ ЧЕГОЛИБО
all_data["Lot_occupation"] = all_data["GrLivArea"] / all_data[
    "LotArea"]  # Отношение площади занимаемой домом ко всей площади участка
all_data["Number_of_floors"] = (all_data[["TotalBsmtSF", "1stFlrSF", "2ndFlrSF"]] != 0).sum(axis=1)  # НАЛИЧИЕ ЧЕГОЛИБО
all_data['Has_2ndFlr'] = all_data['2ndFlrSF'].notnull()  # НАЛИЧИЕ ЧЕГОЛИБО
all_data['Area_Quality_Indicator'] = all_data['OverallQual'] * (all_data['GrLivArea'] + all_data['TotalBsmtSF']) + (
        all_data['WoodDeckSF'] + all_data['OpenPorchSF'] + all_data['EnclosedPorch'] + all_data['3SsnPorch'] +
        all_data['ScreenPorch'])
all_data['Area_Qual_Cond_Indicator'] = all_data['Area_Quality_Indicator'] * all_data['OverallCond']  # СОСТОЯНИЕ
all_data["Has_garage"] = all_data["GarageYrBlt"].notnull()  # НАЛИЧИЕ ЧЕГОЛИБО
all_data["Is_Remodeled"] = (all_data["YearBuilt"] != all_data["YearRemodAdd"])  # СОСТОЯНИЕ
all_data['HasBsmt'] = all_data['BsmtQual'].notnull()  # НАЛИЧИЕ ЧЕГОЛИБО
all_data['Quality_conditition'] = all_data['OverallQual'] * all_data['OverallCond']  # СОСТОЯНИЕ

In [627]:
all_data.drop(['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'MasVnrType', 'FireplaceQu', 'LotFrontage'], axis=1, inplace=True)

In [628]:
for col in all_data.columns:
    if all_data[col].dtype != 'object':
        all_data[col] = all_data[col].fillna(all_data[col].median())
    else:
        all_data[col] = all_data[col].fillna(all_data[col].mode()[0])

In [629]:
all_data = pd.get_dummies(all_data)

In [630]:
train_df = all_data.iloc[:1460]
test_df = all_data.iloc[1460:]

In [631]:
train_df = train_df.drop(train_df[(train_df['GrLivArea'] > 4000)
                                  & (train_df['SalePrice'] < 200000)].index)
train_df = train_df.drop(train_df[(train_df['GarageArea'] > 1200)
                                  & (train_df['SalePrice'] < 300000)].index)
train_df = train_df.drop(train_df[(train_df['TotalBsmtSF'] > 4000)
                                  & (train_df['SalePrice'] < 200000)].index)
train_df = train_df.drop(train_df[(train_df['1stFlrSF'] > 4000)
                                  & (train_df['SalePrice'] < 200000)].index)

train_df = train_df.drop(train_df[(train_df['TotRmsAbvGrd'] > 12)
                                  & (train_df['SalePrice'] < 230000)].index)

In [632]:
y = train_df.SalePrice

In [633]:
X = train_df.drop('SalePrice', axis=1)

In [634]:
from sklearn.model_selection import GridSearchCV


model_xgb = xgb.XGBRegressor(
    #n_estimators=2500,
    learning_rate=0.1,
    max_depth=2,
    max_leaves=4,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

model_xgb.fit(X, y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=2, max_leaves=4,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=900, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [636]:
test_df.drop('SalePrice', axis=1, inplace=True)
xgb_preds = model_xgb.predict(test_df)

C:\Users\Daniil\AppData\Local\Temp\ipykernel_15956\1119368150.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.drop('SalePrice', axis=1, inplace=True)


In [637]:
from sklearn.linear_model import RidgeCV

# СЮДА МОЖНО ПОДКИНУТЬ StandardScaler()

ridge_cv = RidgeCV(alphas=11.721)
ridge_cv.fit(X, y)
ridge_cv_preds = ridge_cv.predict(test_df)

In [638]:
submission = {
    'Id': pd.read_csv('data/test.csv').Id.values,
    'SalePrice': ridge_cv_preds * 0.45 + xgb_preds * 0.55
}
solution = pd.DataFrame(submission)
solution.to_csv('submission.csv', index=False)  # 12876, 0.03%

In [639]:
422 / 109671

0.003847872272524186